In [6]:
!pip install ortools

# Worksheet 4: CSP and OR-Tools

This week, you will gain hands-on experience of using the CSP solver from the OR-Tools suite.
"OR-Tools is an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, integer and linear programming, and constraint programming" ([source](https://developers.google.com/optimization)).

OR-Tools is being developed by Google, and it has been consistently winning competitions such as [MiniZinc](https://www.minizinc.org/challenge.html).  The suite includes several solvers for a range of mathematical programs including CP-SAT, which is a CSP solver based on a SAT solver.

## Installation of the Python package

Run the command at the top of the notebook to install OR-Tools.  It may take 10&ndash;20 seconds.

## Exercises
**Exercise 1.**  Study the example below.  It uses OR-Tools to solve problem (1)&ndash;(7):

$$
\begin{split}
& b \rightarrow (x > y), \qquad & (1)\\
& x \neq y,\ x \neq z,\ y \neq z, \qquad & (2)\\
& a \lor b, \qquad & (3)\\
& a = 0, \qquad & (4)\\
& 0 \le x, y, z \le 2, \qquad & (5)\\
& x, y, z \text{ are integers}, \qquad & (6)\\
& a, b \in \{ 0, 1 \}. \qquad & (7)\\
\end{split}
$$





In [ ]:
from ortools.sat.python import cp_model # CP-SAT solver

# Create the 'model', i.e. the knowledge base:
model = cp_model.CpModel()

# Create the variables (this is the CSP version of FOL constants):
x = model.NewIntVar(0, 2, 'x')
y = model.NewIntVar(0, 2, 'y')
z = model.NewIntVar(0, 2, 'z')

a = model.NewBoolVar('a')
b = model.NewBoolVar('b')

# Define the constraints:
model.Add(x > y).OnlyEnforceIf(b)

model.AddAllDifferent([x, y, z])
model.AddBoolOr([a, b]) # (a or b) has to hold
model.Add(a == 0)

# Create a solver and solve the model:
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Check if a solution was found:
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    # Print the solution
    print(f'x = {solver.Value(x)}')
    print(f'y = {solver.Value(y)}')
    print(f'z = {solver.Value(z)}')
    print(f'a = {bool(solver.Value(a))}')
    print(f'b = {bool(solver.Value(b))}')
else:
    print('unsat')

x = 2
y = 0
z = 1
a = False
b = True


**Exercise 2.**  Come up with some modification of the formulation (1)&ndash;(8) that would make it unsatisfiable.  Copy the above code and make the change; verify using OR-Tools that the problem is indeed unsatisfiable.

In [ ]:
# Your solution to exercise 2
from ortools.sat.python import cp_model # CP-SAT solver

# Create the 'model', i.e. the knowledge base:
model = cp_model.CpModel()

# Create the variables (this is the CSP version of FOL constants):
x = model.NewIntVar(0, 2, 'x')
y = model.NewIntVar(0, 2, 'y')
z = model.NewIntVar(0, 2, 'z')

a = model.NewBoolVar('a')
b = model.NewBoolVar('b')

# Define the constraints:
model.Add(x > y).OnlyEnforceIf(b)
model.Add(x < y) #add
model.AddAllDifferent([x, y, z])
model.AddBoolOr([a, b]) # (a or b) has to hold
model.Add(a == 0)

# Create a solver and solve the model:
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Check if a solution was found:
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    # Print the solution
    print(f'x = {solver.Value(x)}')
    print(f'y = {solver.Value(y)}')
    print(f'z = {solver.Value(z)}')
    print(f'a = {bool(solver.Value(a))}')
    print(f'b = {bool(solver.Value(b))}')
else:
    print('unsat')


unsat


**Exercise 3.**  Implement a solver of the Latin Square Problem using OR-Tools.  Use the approach with individual integer constants for each matrix element and explicit enumeration of constraints $m_{i,j} \neq m_{i,k}$, etc. (formulation (5)&ndash;(8) from Worksheet&nbsp;3).  Here is the CSP version of it (it is very similar to the FOL version):
$$
\begin{split}
& m_{i,j} \neq m_{k,j} \qquad & \forall i < k \in N,\ \forall j \in N, \qquad & (8)\\
& m_{i,j} \neq m_{i,k} & \forall i \in N,\ \forall j < k \in N, \qquad & (9)\\
& m_{i,j} = t & \forall (i, j, t) \in V, \qquad & (10)\\
& m_{i,j} \in N & \forall i, j \in N, \qquad & (11)
\end{split}
$$
where $N = \{ 1, 2, \ldots, n \}$.

In [30]:
# Your solution to exercise 3

from ortools.sat.python import cp_model

def solve_latin_square(n, V):

    model = cp_model.CpModel()

    m = [[model.NewIntVar(1, n, f'm_{i}_{j}') for j in range (1,n+1)] for i in range (1, n+1)]

    for i in range (1, n+1):
      for j in range (1, n+1):
        for k in range (j+1, n+1):
          model.Add(m[i-1][j-1] != m[i-1][k-1])

    for j in range (1,n+1):
      for i in range (1,n+1):
        for k in range (i+1, n+1):
          model.Add(m[i-1][j-1] != m[k-1][j-1])


    for i in range (0, len(V)):
      t = V[i]
      model.Add(m[t[0]][t[1]] == t[2])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
      for i in range (1,n+1):
        print("")
        for j in range (1, n+1):
          print (f'{solver.value(m[i-1][j-1]) }' , end = ' ')
    else:
      print('unsat')



solve_latin_square(10, [(1, 1, 2), (2, 2, 3)])


8 6 9 7 5 3 4 2 1 10 
7 2 8 5 4 6 3 1 10 9 
5 4 3 6 7 2 1 10 9 8 
6 10 5 4 3 1 2 9 8 7 
3 9 4 1 2 5 10 8 7 6 
4 8 2 3 1 10 9 7 6 5 
1 3 7 2 10 9 8 6 5 4 
2 1 6 10 9 8 7 5 4 3 
10 5 1 9 8 7 6 4 3 2 
9 7 10 8 6 4 5 3 2 1 

**Exercise 4.**  Compare the performances of your solvers based on Z3 and OR-Tools.  Which one is faster?

Your answer to exercise 4:

 OR-Tools

**Exercise 5.**  CSP has a standard constraint _AllDiff_; it requests that all the variables in a given set are assigned distinct values.
This constraint is particularly convenient in the formulation of the Latin square problem.
Mathematically, we can formulate the Latin square problem as follows.

Let $n$ be the size of the Latin square.
Let $N = \{ 1, 2, \ldots, n \}$ &ndash; we introduce this set for convenience.
Let $V$ be a set of tuples $(i, j, t)$ of known Latin square values, where $i$ and $j$ are the known element coordinates and $t$ is the known element value.
(For the example given in Worksheet&nbsp;3, $V = \big\{ (1, 3, 1),\ (2, 1, 2),\ (3, 3, 3) \big\}$.)

$$
\begin{align}
& \text{AllDiff}(\{ m_{i,j} : i \in N \}) && \forall j \in N, && (12)\\
& \text{AllDiff}(\{ m_{i,j} : j \in N \}) && \forall i \in N, && (13)\\
& m_{i,j} = t && \forall (i, j, t) \in V, && (14)\\
& m_{i,j} \in N && \forall i, j \in N. && (15)
\end{align}
$$

Constraints (12) request that the values are not repeated within each column, constraints (13) request that the values are not repeated within each row and constraints (14) request that the known values are respected.
(15) define variables $m_{i,j}$ for $i, j \in N$ and their domains.

In [33]:
# Your solution to exercise 5

from ortools.sat.python import cp_model

def solve_latin_square_alldiff(n, V):
    model = cp_model.CpModel()


    m = [[model.NewIntVar(1, n, f'm_{i}_{j}') for j in range (1,n+1)] for i in range (1, n+1)]

    for i in range (1, n+1):
      model.AddAllDifferent(m[i-1])

    for j in range (1,n+1):
      model.AddAllDifferent(m[i-1][j-1] for i in range (n))

    for i in range (0, len(V)):
      t = V[i]
      model.Add(m[t[0]-1][t[1]-1] == t[2])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
      for i in range (1,n+1):
        print("")
        for j in range (1, n+1):
          print (f'{solver.value(m[i-1][j-1]) }' , end = ' ')
    else:
      print('unsat')

    # Your code here

solve_latin_square_alldiff(10, [(1, 1, 2), (2, 2, 3)])


2 9 6 4 10 5 3 7 8 1 
5 3 9 7 1 2 8 4 6 10 
9 8 3 6 5 10 1 2 7 4 
10 4 2 1 8 7 6 3 9 5 
8 7 1 3 6 4 2 10 5 9 
7 1 4 5 3 6 10 9 2 8 
6 10 8 2 7 9 4 5 1 3 
4 6 5 8 2 3 9 1 10 7 
3 2 7 10 9 1 5 8 4 6 
1 5 10 9 4 8 7 6 3 2 

**Exercise 6.**  Let us introduce a modification of the Latin Square Problem which we will call *Bounded Latin Square Problem*.
The Bounded Latin Square Problem is exactly the same as the Latin Square Problem but it adds one more condition:
$$
\max_{i, j \in \{ 1, \ldots, k \}} m_{i,j} < \min_{i, j \in \{ n - k + 1, \ldots, n \}} m_{i,j}, \qquad (16)
$$
where $k$ is a parameter, $1 \le k < n$.
In other words, the maximum element in the top left submatrix of size $k \times k$ is smaller than the minimum element in the bottom right submatrix of size $k \times k$.
See an example of a solution to the Bounded Latin Square below:

![Image](https://i.ibb.co/zNC88Q7/bounded-latin-square.png)

Here $n = 6$, $k = 2$ and $V = \emptyset$.

Implement a solver for the Bounded Latin Square Problem.
Use the `AddMaxEquality` and `AddMinEquality` functions (see the [documentation](https://developers.google.com/optimization/reference/python/sat/python/cp_model#addmaxequality)).
You may find that the documentation is scarce, but this is common in such specialist libraries, hence it is a useful exercise.

To use these functions, you will need _auxiliary variables_.
For example, constraint `AddMaxEquality` requests that variable `target` is equal to the maximum among the variables `variables`.
While it is clear which variables should be included in the list `variables`, what should be used for `target`?
This has to be a new (auxiliary) variable because we could not use any of the existing variables.

Write down (perhaps, on paper) your formulation of the Bounded Latin Square Problem and implement it in Python/OR Tools.

Ask for help if you get stuck.

In [42]:
# Your solution to exercise 6

from ortools.sat.python import cp_model

def solve_bounded_latin_square(n, m, V):
    model = cp_model.CpModel()
    mar = [[model.NewIntVar(1, n, f'mar_{i}_{j}') for j in range (1,n+1)] for i in range (1, n+1)]

    for i in range (1, n+1):
      model.AddAllDifferent(mar[i-1])

    for j in range (1,n+1):
      model.AddAllDifferent(mar[i-1][j-1] for i in range (n))


    for i in range (0, len(V)):
      t = V[i]
      model.Add(mar[t[0]-1][t[1]-1] == t[2])
    up_list = []
    for i in range (1,m+1):
      for j in range (1, m+1):
        up_list.append(mar[i-1][j-1])
    max = model.NewIntVar(1,n,'max')
    model.AddMaxEquality(max, up_list)

    print (max)

    low_list = []
    for i in range (n-m+1, n+1):
      for j in range (n-m+1, n+1):
        low_list.append(mar[i-1][j-1])
    min = model.NewIntVar(1,n,'min')
    model.AddMinEquality(min, low_list)


    model.Add(max<min)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
      for i in range (1,n+1):
        print("")
        for j in range (1, n+1):
          print (f'{solver.value(mar[i-1][j-1]) }' , end = ' ')
    else:
      print('unsat')
    # Your code here

solve_bounded_latin_square(10, 2, [(1, 1, 2), (2, 2, 3)])

max

2 4 6 3 9 5 8 7 1 10 
4 3 8 10 2 6 7 1 5 9 
8 7 3 2 5 1 6 9 10 4 
7 6 1 4 8 2 10 5 9 3 
6 1 5 9 10 4 2 8 3 7 
3 8 7 5 1 10 9 2 4 6 
5 9 10 8 4 7 3 6 2 1 
10 5 9 1 7 8 4 3 6 2 
1 2 4 6 3 9 5 10 7 8 
9 10 2 7 6 3 1 4 8 5 

**Exercise 7.**  It is actually very easy to get rid of the `AddMaxEquality` and `AddMinEquality` functions.
Modify your solver so that it does not use those functions.
Update the formulation accordingly.

Can you think of a way to formulate the Bounded Latin Square Problem with just one auxiliary variable?

In fact, there is a way to formulate it without auxiliary variables, but it is not particularly compact.
As an optional exercise, you may want to implement such a formulation and compare its performance to the performance of the formulation with auxiliary variables.

In [44]:
# Your solution to exercise 7
from ortools.sat.python import cp_model

def solve_bounded_latin_square(n, m, V):
    model = cp_model.CpModel()
    mar = [[model.NewIntVar(1, n, f'mar_{i}_{j}') for j in range (1,n+1)] for i in range (1, n+1)]

    for i in range (1, n+1):
      model.AddAllDifferent(mar[i-1])

    for j in range (1,n+1):
      model.AddAllDifferent(mar[i-1][j-1] for i in range (n))


    for i in range (0, len(V)):
      t = V[i]
      model.Add(mar[t[0]-1][t[1]-1] == t[2])
    up_list = []
    for i in range (1,m+1):
      for j in range (1, m+1):
        up_list.append(mar[i-1][j-1])
    max = model.NewIntVar(1,n,'max')

    for i in range (len(up_list)):
      model.Add(max >= up_list[i])


    low_list = []
    for i in range (n-m+1, n+1):
      for j in range (n-m+1, n+1):
        low_list.append(mar[i-1][j-1])
    min = model.NewIntVar(1,n,'min')

    for i in range (len(low_list)):
      model.Add(min <= low_list[i])



    model.Add(max<min)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
      for i in range (1,n+1):
        print("")
        for j in range (1, n+1):
          print (f'{solver.value(mar[i-1][j-1]) }' , end = ' ')
    else:
      print('unsat')
    # Your code here

solve_bounded_latin_square(10, 2, [(1, 1, 2), (2, 2, 3)])



2 1 9 4 6 5 10 8 3 7 
1 3 6 9 5 2 4 10 7 8 
10 7 4 8 3 6 5 2 9 1 
3 10 5 2 9 1 7 4 8 6 
6 4 8 7 10 9 2 5 1 3 
8 6 10 5 4 3 1 7 2 9 
5 9 3 1 7 4 8 6 10 2 
7 5 1 3 2 8 6 9 4 10 
9 2 7 6 8 10 3 1 5 4 
4 8 2 10 1 7 9 3 6 5 

**Exercise 8.**  Let us introduce one more modification of the problem.
We will call it Conditional Bounded Latin Square Problem.
I will not explain it in any way other than the mathematical formulation:
$$
\begin{split}
& \begin{array}
\text{\text{AllDiff}}(\{ m_{i,j} : i \in N \}) & \forall j \in N, \qquad \qquad \qquad \qquad \qquad \quad & (17)\\
\text{AllDiff}(\{ m_{i,j} : j \in N \}) & \forall i \in N, & (18) \\
m_{i,j} = v & \forall (i, j, v) \in V, & (19) \\
m_{i,j} \in N & \forall i, j \in N, & (20) \\
\end{array}\\
& \displaystyle{\left(\max_{i, j \in \{ 1, \ldots, k \}} m_{i,j} \le k \right)~~ \text{or} ~~\left( \min_{i, j \in \{ n - k + 1, \ldots, n \}} m_{i,j} > n - k \right). \quad (21)}
\end{split}
$$

To implement this, you might want to use the [`OnlyEnforceIf`](https://developers.google.com/optimization/reference/python/sat/python/cp_model#onlyenforceif) function of the `Constraint` class.

In [45]:
# Your solution to exercise 8

from ortools.sat.python import cp_model

def solve_conditional_bounded_latin_square(n, m, V):
    model = cp_model.CpModel()

    mar = [[model.NewIntVar(1, n, f'mar_{i}_{j}') for j in range (1,n+1)] for i in range (1, n+1)]

    for i in range (1, n+1):
      model.AddAllDifferent(mar[i-1])

    for j in range (1,n+1):
      model.AddAllDifferent(mar[i-1][j-1] for i in range (n))


    for i in range (0, len(V)):
      t = V[i]
      model.Add(mar[t[0]-1][t[1]-1] == t[2])

    up_list = []
    for i in range (1,m+1):
      for j in range (1, m+1):
        up_list.append(mar[i-1][j-1])
    max = model.NewIntVar(1,n,'max')
    model.AddMaxEquality(max, up_list)


    low_list = []
    for i in range (n-m+1, n+1):
      for j in range (n-m+1, n+1):
        low_list.append(mar[i-1][j-1])
    min = model.NewIntVar(1,n,'min')
    model.AddMinEquality(min, low_list)

    a = model.NewBoolVar('a')
    b = model.NewBoolVar('b')

    model.Add(max <= m).OnlyEnforceIf(a)
    model.Add(min > (n-m)).OnlyEnforceIf(b)

    model.AddBoolOr([a,b])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
      for i in range (1,n+1):
        print("")
        for j in range (1, n+1):
          print (f'{solver.value(mar[i-1][j-1]) }' , end = ' ')
    else:
      print('unsat')

    # Your code here

solve_conditional_bounded_latin_square(10, 2, [(1, 1, 2), (2, 2, 3)])


2 6 1 7 4 9 10 8 3 5 
4 3 10 1 2 5 9 7 6 8 
10 9 6 2 8 3 1 5 4 7 
3 5 2 4 1 10 7 9 8 6 
6 8 5 9 10 1 3 2 7 4 
8 4 9 5 6 7 2 10 1 3 
9 7 4 10 3 6 8 1 5 2 
5 10 7 3 9 8 6 4 2 1 
1 2 8 6 7 4 5 3 9 10 
7 1 3 8 5 2 4 6 10 9 

**Warning.** The `OnlyEnforceIf` function is only implemented for linear constraints (the `Add` function), `AddBoolOr` and `AddBoolAnd`.  If you use it on a constraint of a different type such as `AddAllDifferent` then it will be silently ignored by OR-Tools.  (Questionable design choice by Google :) )